In [ ]:
# import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("mode.chained_assignment", None)

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

import tkinter as tk
from tkinter import filedialog

import statsmodels.api as sm
# from statsmodels.formula.api import ols
# from statsmodels.compat import lzip
# from __future__ import print_function

$$ Price \ Elasticity \ of \ Demand = \frac{\% \ Change \ in \ \color{red}{Quantity} \ \color{white}{Demand} \ of \ Product \ \color{red}{A}}{\% \ Change \ in \ \color{red}{Price} \ \color{white}{of} \ Product \ \color{red}{A}} $$
$$ Cross \ Price \ Elasticity \ of \ Demand = \frac{\% \ Change \ in \ \color{red}{Quantity} \ \color{white}{Demand} \ of \ Product \ \color{red}{A}}{\% \ Change \ in \ \color{red}{Price} \ \color{white}{of} \ Product \ \color{red}{B}} $$

#### **Competitors/substitutes:**
We would have a **positive** cross-price correlation:
> ## When one product <font color="red">decreases</font> price, another one's quantity sold <font color="red">decreases</font>

#### **Complements:**
Getting a **negtive** cross-price correlation:
> ## When one product <font color="red">decreases</font> price, another one's quantity sold <font color="red">increases</font>



In [ ]:
root = tk.Tk()
root.withdraw()
path = filedialog.askopenfilename()

In [ ]:
df = pd.read_csv(path, encoding="ISO-8859-1")
df_tbr = df[(df["BRIDGESTONE TYPE"] == "TBR") & (df["DIMENSION (Truck Tyres)"].str.contains("315/80/22.5")) & (df["BRAND"].isin(["MAXXIS", "MICHELIN", "BRIDGESTONE"]))]

In [ ]:
df_tbr.columns

In [ ]:
df_tbr["Product"] = df_tbr["BRAND"] + " " + df_tbr["Item"] + " " + df_tbr["DIMENSION (Truck Tyres)"]
test1 = df_tbr.groupby(["Period", "Product"]).agg({"PRICE TWD/UN.": "mean", "SALES UNITS": "sum"}).reset_index()
x_pivot = test1.pivot(index="Period", columns="Product", values="PRICE TWD/UN.")
y = test1.pivot(index="Period", columns="Product", values="SALES UNITS")


In [ ]:
x_values = pd.DataFrame(x_pivot.to_records())
x_values.fillna(method="ffill", inplace=True)
x_values

In [ ]:
y = y[["BRIDGESTONE R150 315/80/22.5"]]
y.columns = ["R150 sales"]
y = y.reset_index()
y

In [ ]:
result = pd.merge(x_values, y, on="Period")

In [ ]:
result.dropna(inplace=True)

In [ ]:
x_values = result.loc[:, result.columns[1:-1]]
y = result.loc[:, result.columns[-1]]

In [ ]:
mean_xvalues = x_values.mean()
mean_yvalues = y.mean()

In [ ]:
X = sm.add_constant(x_values)
model = sm.OLS(y, X)
result = model.fit()

results_summry = result.summary()
results_summry

In [ ]:
result.pvalues